1. Create a subclass of `jet.Gate` which represents a fully-parameterized unitary.

In [55]:
import jet
import numpy as np

import UnitaryPreperation

class Unitary(jet.Gate):
    """Unitary represents an arbitrary unitary gate.

    Args:
        num_wires (int): Number of wires the gate is applied to.
        params (array): Matrix of gate parameters.
    """

    def __init__(self, num_wires: int, params: np.ndarray):
        super().__init__(name="Unitary", num_wires=num_wires, params=params)

    def _data(self) -> np.ndarray:
        return UnitaryPreperation.param_unitary(dim=2**self.num_wires, params=self.params)

2. Define the parameters of the circuit using the code in `TensorNetworks_QML_final.ipynb`.

In [56]:
# Defining the parameters of the circuit
depth = 3
n_qubits = 16
loc = 4

# depth = 4
# n_qubits = 64
# loc = 8

num_gates = 2**depth-1

# Generating random parameters for the gates
params = []
for i in range(num_gates):
    params.append(np.random.rand(2**loc, 2**loc))

3. Append the gates to a `Circuit`.

In [57]:
circuit = jet.Circuit(dim=2, num_wires=n_qubits)

# Qubits are initialized to |0> but we can easily change this. For example:
# for op in circuit.operations:
    # op.part._state_vector = np.array([0, 1], dtype=np.complex128)

param_index = 0
for layer in UnitaryPreperation.compute_indices(loc, depth):
    for wire_ids in layer:
        gate = Unitary(num_wires=loc, params=params[param_index])
        circuit.append_gate(gate, wire_ids=wire_ids)
        param_index += 1

# Close the circuit with an observable.
observable = [jet.Operation(part=jet.PauliZ(), wire_ids=[n_qubits - 1])]
circuit.observe(observable)

4. Convert the `Circuit` into a `TensorNetwork`.

In [58]:
# Currently, Jet supports 64-bit and 128-bit complex data types.
dtype = np.dtype(np.complex64)

tn = circuit.tensor_network(dtype=dtype)

print(f"Loaded tensor network with {tn.num_tensors} tensors and {tn.num_indices} indices.")

Loaded tensor network with 159 tensors and 305 indices.


5. Serialize the `TensorNetwork` to JSON (if desired).

In [59]:
tnf_str = jet.TensorNetworkSerializer(dtype=dtype)(tn)

filename = "tnf.json"
with open(filename, "w") as f:
    f.write(tnf_str)

print(f"Created tensor network file '{filename}'.")

Created tensor network file 'tnf.json'.


6. Find a suitable contraction path and wrap it in a `PathInfo` instance.

In [60]:
import random

def sample_contraction_path(tn: jet.TensorNetworkType) -> jet.PathInfo:
    path = []

    # Caching the index-to-edge map improves performance by 50x.
    index_to_edge_map = tn.index_to_edge_map

    # Construct a graph that associates nodes with its neighbouring in the TN.
    graph = {}
    for node in tn.nodes:
        graph[node.id] = set()
        for index in node.indices:
            node_ids = index_to_edge_map[index].node_ids
            graph[node.id].update(node_ids)
        graph[node.id].remove(node.id)

    # Iteratively contract a random edge from the graph.
    while len(graph) > 1:
        node_id_1 = random.choice(tuple(graph))
        node_id_2 = random.choice(tuple(graph[node_id_1]))
        path.append((node_id_1, node_id_2))

        node_id_3 = max(graph) + 1
        graph[node_id_3] = (graph[node_id_1] | graph[node_id_2]) - {node_id_1, node_id_2}

        for node_id in graph[node_id_1]:
            if node_id != node_id_2:
                graph[node_id].remove(node_id_1)
                graph[node_id].add(node_id_3)

        for node_id in graph[node_id_2]:
            if node_id != node_id_1:
                graph[node_id].remove(node_id_2)
                graph[node_id].add(node_id_3)

        del graph[node_id_1]
        del graph[node_id_2]

    return jet.PathInfo(tn=tn, path=path)

def find_contraction_path(tn: jet.TensorNetworkType, samples: int) -> jet.PathInfo:
    paths = (sample_contraction_path(tn) for _ in range(samples))
    return min(paths, key=lambda path: path.total_flops())

path_info = find_contraction_path(tn=tn, samples=100)

# Precomputed path for 8x8 circuit using CoTenGra.
# path_info = jet.PathInfo(tn=tn, path=[(92, 114), (111, 159), (117, 160), (116, 161), (118, 162), (112, 163), (113, 164), (115, 165), (22, 66), (16, 167), (21, 168), (18, 169), (19, 170), (20, 171), (17, 172), (23, 173), (166, 174), (85, 175), (31, 67), (30, 177), (29, 178), (25, 179), (28, 180), (26, 181), (24, 182), (27, 183), (91, 123), (125, 185), (124, 186), (120, 187), (121, 188), (119, 189), (122, 190), (126, 191), (184, 192), (73, 193), (176, 194), (82, 195), (0, 64), (4, 197), (5, 198), (6, 199), (1, 200), (3, 201), (7, 202), (2, 203), (94, 95), (101, 205), (96, 206), (97, 207), (98, 208), (99, 209), (102, 210), (100, 211), (204, 212), (72, 213), (93, 105), (108, 215), (109, 216), (107, 217), (103, 218), (106, 219), (110, 220), (104, 221), (12, 65), (15, 223), (13, 224), (10, 225), (8, 226), (9, 227), (11, 228), (14, 229), (222, 230), (86, 231), (214, 232), (196, 233), (76, 234), (80, 235), (78, 236), (77, 237), (90, 130), (128, 239), (131, 240), (129, 241), (133, 242), (127, 243), (134, 244), (132, 245), (35, 68), (36, 247), (38, 248), (37, 249), (34, 250), (32, 251), (39, 252), (33, 253), (246, 254), (84, 255), (42, 69), (41, 257), (45, 258), (46, 259), (43, 260), (47, 261), (44, 262), (40, 263), (89, 135), (139, 265), (138, 266), (142, 267), (140, 268), (141, 269), (137, 270), (136, 271), (264, 272), (256, 273), (74, 274), (81, 275), (238, 276), (61, 71), (58, 278), (59, 279), (60, 280), (63, 281), (57, 282), (56, 283), (62, 284), (79, 285), (87, 157), (154, 287), (152, 288), (158, 289), (155, 290), (153, 291), (151, 292), (156, 293), (286, 294), (75, 295), (277, 296), (83, 297), (88, 143), (144, 299), (147, 300), (150, 301), (146, 302), (145, 303), (149, 304), (148, 305), (298, 306), (70, 307), (55, 308), (48, 309), (51, 52), (54, 311), (49, 53), (50, 313), (312, 314), (310, 315)])

# The memory reported by a contraction path assumes single-byte tensor elements.
num_steps = len(path_info.steps)
num_bytes = int(path_info.total_memory()) * dtype.itemsize
print(f"Loaded contraction path with {num_steps} steps that uses {num_bytes / 10**6:.1f}MB of memory.")

Loaded contraction path with 317 steps that uses 30.4MB of memory.


7. Contract the `TensorNetwork` using the task-based contractor.

In [61]:
import timeit

tbc = jet.TaskBasedContractor(dtype=dtype)
tbc.add_contraction_tasks(tn=tn, path_info=path_info)
tbc.add_deletion_tasks()

print("Starting to contract tensor network. This can take a while...")
t0 = timeit.default_timer()
tbc.contract()
t1 = timeit.default_timer()

result = tbc.results[0].scalar

# Display the contraction result and duration.
print(f"Got contraction result {result} in {t1 - t0:.3f}s.")

Starting to contract tensor network. This can take a while...
Got contraction result (0.023864932358264923+1.469743438065052e-09j) in 0.040s.
